In [15]:
import numpy as np
import cv2
import mediapipe as mp
from tensorflow.keras.models import load_model
import warnings
import h5py
from sklearn.preprocessing import LabelEncoder 
import numpy as np
from sklearn.preprocessing import LabelEncoder 
import numpy as np
warnings.filterwarnings('ignore', category=UserWarning)

In [16]:

def load_data(h5_filepath):
    with h5py.File(h5_filepath, 'r') as hf:
        Y = np.array([cadena.decode('utf-8')for cadena in hf['Y'][:]])
    return Y

Y=load_data("datos.h5")

In [17]:
label_encoder = LabelEncoder()
label_encoder.fit(Y)

LabelEncoder()

In [18]:

model = load_model('modelo.keras')


In [19]:
def calcular_bounding_box(x_min,x_max,y_min,y_max, width, height, margen=0.1):
    box_width = x_max - x_min
    box_height = y_max - y_min
    x_min = max(0, x_min - margen * box_width)
    x_max = min(width, x_max + margen * box_width)
    y_min = max(0, y_min - margen * box_height)
    y_max = min(height, y_max + margen * box_height)
    
    return int(x_min), int(x_max), int(y_min), int(y_max)

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

modelo_landmarks = mp.solutions.drawing_utils  # Will detect 21 hand points
modelo_manos = mp.solutions.hands

# Use webcam (replace 0 with a video file path for video input)
video = cv2.VideoCapture(0)#("./videos/Y.mp4")

# Initialize hand detection model
with modelo_manos.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.7
) as hands:
    while True:
        r, image = video.read()
        if r is None:
            break
        
        image = cv2.resize(image, (640, 480))
        height, width, _ = image.shape

        image = cv2.flip(image, 1)  # Flip image for mirror effect
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Detect hands
        results = hands.process(image_rgb)
 
        if results.multi_hand_landmarks is not None:
            # Process each detected hand
            for marcador_manos in results.multi_hand_landmarks:
                landmarks = []
                X = []
                x_min, x_max = 999, -999
                y_min, y_max = 999, -999

                # Collect hand landmarks and calculate bounding box
                for punto in marcador_manos.landmark:
                    landmarks += [punto.x, punto.y]  # Append x, y coordinates
                    x_min = min(punto.x, x_min)
                    x_max = max(punto.x, x_max)
                    y_min = min(punto.y, y_min)
                    y_max = max(punto.y, y_max)

                X.append(landmarks)
                X = np.array(X)
                
                # Calculate box coordinates in pixels
                x_min, x_max, y_min, y_max = (
                    int(x_min * width), int(x_max * width),
                    int(y_min * height), int(y_max * height)
                )

                # Make prediction (replace with your model's prediction code)
                pred = np.argmax(model.predict(X))
                label = label_encoder.inverse_transform([pred])[0]

                # Define the space below the box in pixels (0.5 cm ≈ 19 pixels)
                label_offset = 19

                # Draw bounding box around hand
                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

                # Center the label below the bounding box with 0.5 cm offset
                label_x = x_min + (x_max - x_min) // 2
                label_y = y_max + label_offset
                cv2.putText(image, label, (label_x, label_y), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

                # Draw landmarks and connections
                modelo_landmarks.draw_landmarks(
                    image, marcador_manos, modelo_manos.HAND_CONNECTIONS,
                    modelo_landmarks.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=4),
                    modelo_landmarks.DrawingSpec(color=(0, 255, 0), thickness=2)
                )

        cv2.imshow("Hand Recognition", image)

        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        image = cv2.flip(image, 1)

video.release()
cv2.destroyAllWindows()
